In [1]:
import numpy as np
import pandas as pd
import datetime
import matplotlib 
from matplotlib import pyplot as plt
import gc
import multiprocessing as mp
from multiprocessing import Process, Queue
queue = Queue()
import random
from matplotlib.pyplot import figure
import math

In [2]:
def read_df(file_title,house_nb):
    try:
        df=pd.read_table(file_title, sep="," )
        df.columns = ["time","consumption"]
        try:
            df['time'] = pd.to_datetime(df['time'],unit='s')
        except:
            df['time'] = pd.to_datetime(df['time'])
        df.index= df.time
        df = df.sort_index()
        df.reset_index(drop=True, inplace=True)
        return (df,house_nb)
    except:
        return(None)

In [3]:
pool = mp.Pool(processes = 3)
result=pool.starmap(read_df,(("Refit_Resampled/fridge_freezer/house_"+str(i+1)+".csv",i+1) for i in range(21)))
Refit_fridge_freezers={}
for i in range(len(result)):
    if result[i]!=None:
        Refit_fridge_freezers[result[i][1]]=result[i][0]

In [4]:
pool = mp.Pool(processes = 3)
result=pool.starmap(read_df,(("Refit_Resampled/fridge/house_"+str(i+1)+".csv",i+1) for i in range(21)))
Refit_fridges={}
for i in range(len(result)):
    if result[i]!=None:
        Refit_fridges[result[i][1]]=result[i][0]

In [5]:
def csv_to_activations(file_title):
    df=pd.read_table(file_title, sep="," )
    df = df[['beginning', 'end','label']]
    activation = [tuple(x[0:2]) for x in df.to_numpy()]
    activation_type = [x[2] for x in df.to_numpy()]
    return(activation,activation_type)

In [6]:
Refit_fridge_freezer_activation_type={}
Refit_fridge_freezer_activations={}
for house_nb in range(22):
    try:
        Refit_fridge_freezer_activations[house_nb]=csv_to_activations("Refit_activations/labels/fridge_freezer_revertBack/house_"+str(house_nb))[0]
        Refit_fridge_freezer_activation_type[house_nb]=csv_to_activations("Refit_activations/labels/fridge_freezer_revertBack/house_"+str(house_nb))[1]
    except:
        None

In [8]:
Refit_fridge_activation_type={}
Refit_fridge_activations={}
for house_nb in range(22):
    try:
        Refit_fridge_activations[house_nb]=csv_to_activations("Refit_activations/labels/fridge_revertBack/house_"+str(house_nb))[0]
        Refit_fridge_activation_type[house_nb]=csv_to_activations("Refit_activations/labels/fridge_revertBack/house_"+str(house_nb))[1]
    except:
        None

In [9]:
def normal_anomolous(activations,activation_type,house_nb):
    normal_activations=[]
    anomolous_activations=[]
    for i in range(len(activations)):
        if activation_type[i]==0:
            normal_activations.append(activations[i])
        else:
            anomolous_activations.append(activations[i])
    return(normal_activations,anomolous_activations,house_nb)

In [10]:
Refit_fridge_freezer_normal_activations={}
Refit_fridge_freezer_anomolous_activations={}
for house_nb in Refit_fridge_freezer_activations.keys():
    result=normal_anomolous(Refit_fridge_freezer_activations[house_nb],Refit_fridge_freezer_activation_type[house_nb],house_nb)
    Refit_fridge_freezer_normal_activations[house_nb]=result[0]
    Refit_fridge_freezer_anomolous_activations[house_nb]=result[1]

In [11]:
Refit_fridge_normal_activations={}
Refit_fridge_anomolous_activations={}
for house_nb in Refit_fridge_activations.keys():
    result=normal_anomolous(Refit_fridge_activations[house_nb],Refit_fridge_activation_type[house_nb],house_nb)
    Refit_fridge_normal_activations[house_nb]=result[0]
    Refit_fridge_anomolous_activations[house_nb]=result[1]

In [12]:
def oversampled_anomalies(normal_activations,anomolous_activations):
    normal_nb=len(normal_activations)
    abnormal_nb=len(anomolous_activations)
    oversampling_nb=normal_nb-abnormal_nb
    oversampled_list=anomolous_activations
    for i in range(0,oversampling_nb):
        oversampled_list.append(random.choice(anomolous_activations))
    oversampled_list.sort()
    return (oversampled_list)

In [13]:
oversampled_anomalies_fridges={}
oversampled_anomalies_fridge_freezers={}
for house in range(0,22):
    try:
        oversampled_anomalies_fridge_freezers[house]=oversampled_anomalies(Refit_fridge_freezer_normal_activations[house],Refit_fridge_freezer_anomolous_activations[house])
    except:
        None
for house in range(0,22):
    try:        
        oversampled_anomalies_fridges[house]=oversampled_anomalies(Refit_fridge_normal_activations[house],Refit_fridge_anomolous_activations[house])
    except:
            None

In [14]:
def get_new_activations(normal_activations,anomolous_activations):
    new_activations=anomolous_activations
    new_activations.extend(normal_activations)
    new_activations.sort()
    return(new_activations)

In [15]:
new_activations_fridges={}
new_activations_fridge_freezers={}
for house in range(0,22):
    try:
        new_activations_fridge_freezers[house]=get_new_activations(Refit_fridge_freezer_normal_activations[house],oversampled_anomalies_fridge_freezers[house])
    except:
        None
for house in range(0,22):
    try:        
        new_activations_fridges[house]=get_new_activations(Refit_fridge_normal_activations[house],oversampled_anomalies_fridges[house])
    except:
            None

In [16]:
def get_activation_type(activations,normal_activations,house_nb):
    activation_type=[]
    for activation in activations:
        if activation in normal_activations:
            activation_type.append(0)
        else:
            activation_type.append(1)
    return(activation_type,house_nb)

In [17]:
pool = mp.Pool(processes = 3)
result=pool.starmap(get_activation_type,((new_activations_fridge_freezers[i],Refit_fridge_freezer_normal_activations[i],i) for i in new_activations_fridge_freezers.keys() ))
Refit_fridge_freezer_activation_type={}
for i in range(len(result)):
        Refit_fridge_freezer_activation_type[result[i][1]]=result[i][0]

In [18]:
pool = mp.Pool(processes = 3)
result=pool.starmap(get_activation_type,((new_activations_fridges[i],Refit_fridge_normal_activations[i],i) for i in new_activations_fridges.keys() ))
Refit_fridge_activation_type={}
for i in range(len(result)):
        Refit_fridge_activation_type[result[i][1]]=result[i][0]

In [19]:
def activations_to_csv(activations,activation_type,house_nb):
    df=pd.DataFrame(activations, columns =['beginning','end'])
    df1 = pd.DataFrame(activation_type, columns =['label'])
    df_row = pd.concat([df, df1],axis=1)
    df_row.to_csv("Refit_activations/labels/fridge_freezer_resampled/house_"+str(house_nb),index=False)

In [20]:
for i in Refit_fridge_freezer_activation_type.keys():
    activations_to_csv(new_activations_fridge_freezers[i],Refit_fridge_freezer_activation_type[i],i)

In [21]:
def activations_to_csv(activations,activation_type,house_nb):
    df=pd.DataFrame(activations, columns =['beginning','end'])
    df1 = pd.DataFrame(activation_type, columns =['label'])
    df_row = pd.concat([df, df1],axis=1)
    df_row.to_csv("Refit_activations/labels/fridge_resampled/house_"+str(house_nb),index=False)

In [22]:
for i in Refit_fridge_activation_type.keys():
    activations_to_csv(new_activations_fridges[i],Refit_fridge_activation_type[i],i)